In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
datagen = ImageDataGenerator(
    rescale=1./255,  # normalize pixel values to [0,1]
    rotation_range=20,  # randomly rotate images
    width_shift_range=0.2,  # randomly shift images horizontally
    height_shift_range=0.2,  # randomly shift images vertically
    shear_range=0.2,  # randomly apply shear transformation
    zoom_range=0.2,  # randomly zoom in and out on images
    horizontal_flip=True,  # randomly flip images horizontally
    fill_mode='nearest'  # fill in newly created pixels
)

def get_label(filename):
    if filename.startswith("homer"):
        return 'homer'
    else:
        return 'bart'

image_filenames = os.listdir('homer_bart_1')
labels = [get_label(filename) for filename in image_filenames]

train_filenames, test_filenames, train_labels, test_labels = train_test_split(image_filenames, labels, test_size=0.2, random_state=42)


train_df = pd.DataFrame({"filename": train_filenames, "label": train_labels})
test_df = pd.DataFrame({"filename": test_filenames, "label": test_labels})


train_generator = datagen.flow_from_dataframe(
    train_df,
    directory='homer_bart_1',
    x_col="filename",
    y_col="label",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_dataframe(
    test_df,
    directory='homer_bart_1',
    x_col="filename",
    y_col="label",
    target_size=(150, 150), 
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42
)

Found 215 validated image filenames belonging to 2 classes.
Found 54 validated image filenames belonging to 2 classes.


In [3]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid')
])

In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       

In [5]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
)

Epoch 1/20


7/7 [==============================] - 3s 258ms/step - loss: 1.0067 - accuracy: 0.5256 - val_loss: 0.7422 - val_accuracy: 0.5741
Epoch 2/20
7/7 [==============================] - 3s 496ms/step - loss: 0.6469 - accuracy: 0.6093 - val_loss: 0.6460 - val_accuracy: 0.6667
Epoch 3/20
7/7 [==============================] - 5s 652ms/step - loss: 0.6081 - accuracy: 0.6140 - val_loss: 0.6107 - val_accuracy: 0.7037
Epoch 4/20
7/7 [==============================] - 5s 642ms/step - loss: 0.5457 - accuracy: 0.7209 - val_loss: 0.6458 - val_accuracy: 0.6111
Epoch 5/20
7/7 [==============================] - 5s 647ms/step - loss: 0.5007 - accuracy: 0.7860 - val_loss: 0.6636 - val_accuracy: 0.6481
Epoch 6/20
7/7 [==============================] - 5s 645ms/step - loss: 0.4199 - accuracy: 0.8047 - val_loss: 0.6215 - val_accuracy: 0.7407
Epoch 7/20
7/7 [==============================] - 5s 646ms/step - loss: 0.3859 - accuracy: 0.8372 - val_loss: 0.5490 - val_accuracy: 0.7407
Epoch 8/20
7/7 [==